In [1]:
import math
import numpy as np
import os
from collections import defaultdict
import scipy.stats as stats
import matplotlib.pyplot as plt
import random as nd
import csv
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Data :
    def __init__(self, inputfile) :
        self.data = self.construct_array(inputfile)
        self.num_games = self.data[-1][0]

    def construct_array (self, inputfile) :
        data = open(inputfile, 'r')
        ndtype = [('game-number', 'int'),
                  ('clock', 'int'),
                  ('high-payoff-location', 'a10'),
                  ('player', 'a5'),
                  ('xcor', 'f8'),
                  ('ycor', 'f8'),
                  ('heading', 'f8'),
                  ('points-earned', 'int'),
                  ('noise', 'f8')]
        strucArray = genfromtxt(data, dtype = ndtype, delimiter = ',')
        return strucArray

    def get_data (self) :
        return self.data

In [3]:
def get_game_data (data, game_number) :
    b = data['gamenumber'] == game_number
    return data[b]

In [4]:
import string
def id_generator(length=8, chars=string.ascii_lowercase + string.digits):
    return ''.join(nd.choice(chars) for i in range(length))

In [5]:
print(nd.randint(1,5))

3


# NEW CODE

In [6]:
def get_big_list(conflict_level, who, cond) :
    max_num_games = 50 if conflict_level == 'high' else 60
    round_n = 0
    big_list = []
    for r,d,f in os.walk('../data/{0}_{1}_{2}/completed'.format(conflict_level, who, cond)):
        for files in f:
            if files.startswith('game_'):
                #file_name = files
                destination = r + '/' + files
                d = Data(destination).get_data()
                round_n += 1
                g = get_game_data(d,1)
                length = g.shape[0]
                for j in range(length):
                    onerow = g[j]
                    onerow[0] = round_n  
                    #print onerow[0]                      
                    big_list.append(onerow)

    with open('BIGLIST_reactive.csv', 'wb') as myfile:
        writer = csv.writer(myfile)
        writer.writerows(big_list)

In [7]:
get_big_list("high", "human", "dynamic")

In [8]:
def get_human_reactive(conflict_level) :
    max_num_games = 50 if conflict_level == 'high' else 60
    file_name = "game_"+id_generator(8)+"-"+id_generator(4)+"-"+id_generator(4)+"-"+id_generator(4)+"-"+id_generator(12)+".csv"
    round_n = 0
    prev_blue = 0
    prev_red = 0
    ts = []
    for r,d,f in os.walk('./reactive_csv_test'):
        for files in f:
            if files.startswith('BIGLIST_'):
                #file_name = files
                destination = r + '/' + files
                d = Data(destination).get_data()
                for i in range(max_num_games):
                #if (round2_n <= max_num_games):
                    round_n += 1
                    biglist_game_n = d[-1][0]
                    rnd = nd.randint(1,biglist_game_n)
                    g = get_game_data(d,rnd)
                    length = g.shape[0]
                    #print g
                    # guarda el score real (no el acumulado) obtenido esta ronda
                    blue_rscore = g[-2][7] - g[0][7]
                    red_rscore = g[-1][7]- g[1][7]                    
                    #print "bluescore", g[-2][7] - g[0][7]
                    #print "redscore", g[-1][7]- g[1][7]
                    # recorre la ronda e iguala los todos los scores al valor de la ronda anterior
                    for j in range(length):
                        if g[j][3] == 'host': g[j][7] = prev_blue
                        elif g[j][3] == 'other': g[j][7] = prev_red
                    # cambia sólo el último valor de cada jugador por el valor obtenido en ésa ronda
                    g[-2][7] += blue_rscore
                    g[-1][7] += red_rscore
                    # actualiza el score acumulado para la siguiente ronda
                    prev_blue += blue_rscore
                    prev_red += red_rscore
                    #print g
                    for j in range(length):
                        onerow = g[j]
                        onerow[0] = round_n  
                        #print onerow[0]                      
                        ts.append(onerow)
            if file_name.endswith('.csv'):
                file_name = file_name[:-4]
                with open(file_name+'_reactive.csv', 'wb') as myfile:
                    writer = csv.writer(myfile)
                    writer.writerows(ts)

In [9]:
get_human_reactive("high")

In [10]:
for i in range(56):
    get_human_reactive("high")